In [1]:
import os
import numpy as np
import pandas as pd
import gc
from xgboost import XGBClassifier

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

pd.options.display.max_rows = 5000
pd.options.display.max_columns = 100

## Small dataset

In [2]:
X, y = make_classification(n_samples=10_000,
                           n_features=20,
                           n_informative=10,
                           n_redundant=5,
                           n_repeated=0,
                           n_classes=2,
                           n_clusters_per_class=4,
                           flip_y=0.03,
                           class_sep=0.5,
                           shuffle=True,
                           random_state=0)

In [3]:
# create a train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=7)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)
del X, y
gc.collect()

40

In [4]:
X_train.shape

(7000, 20)

### XGB exact

In [5]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'exact'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)
del xgb
gc.collect()

[0]	validation_0-auc:0.764968	validation_1-auc:0.701021	validation_2-auc:0.692161
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.817387	validation_1-auc:0.761827	validation_2-auc:0.722534
[2]	validation_0-auc:0.850106	validation_1-auc:0.773021	validation_2-auc:0.74608
[3]	validation_0-auc:0.872547	validation_1-auc:0.795034	validation_2-auc:0.768927
[4]	validation_0-auc:0.882577	validation_1-auc:0.806465	validation_2-auc:0.778654
[5]	validation_0-auc:0.894369	validation_1-auc:0.810276	validation_2-auc:0.791015
[6]	validation_0-auc:0.900538	validation_1-auc:0.813313	validation_2-auc:0.797822
[7]	validation_0-auc:0.902598	validation_1-auc:0.81434	validation_2-auc:0.799095
[8]	validation_0-auc:0.912228	validation_1-auc:0.827051	validation_2-auc:0.806854
[9]	validation_0-auc:0.917469	validation_1-auc:0.833026	validation_2-auc:0.80944
[10]	validation_0-auc:0.91

[98]	validation_0-auc:0.992476	validation_1-auc:0.887811	validation_2-auc:0.879905
[99]	validation_0-auc:0.992701	validation_1-auc:0.887866	validation_2-auc:0.880387
[100]	validation_0-auc:0.992751	validation_1-auc:0.888034	validation_2-auc:0.880449
[101]	validation_0-auc:0.992802	validation_1-auc:0.887945	validation_2-auc:0.880243
[102]	validation_0-auc:0.992991	validation_1-auc:0.887729	validation_2-auc:0.880733
[103]	validation_0-auc:0.993161	validation_1-auc:0.887639	validation_2-auc:0.880835
[104]	validation_0-auc:0.993381	validation_1-auc:0.888342	validation_2-auc:0.881431
[105]	validation_0-auc:0.993522	validation_1-auc:0.888941	validation_2-auc:0.882085
[106]	validation_0-auc:0.993822	validation_1-auc:0.889182	validation_2-auc:0.881818
[107]	validation_0-auc:0.994062	validation_1-auc:0.889039	validation_2-auc:0.881674
[108]	validation_0-auc:0.994212	validation_1-auc:0.888988	validation_2-auc:0.882254
[109]	validation_0-auc:0.994336	validation_1-auc:0.889109	validation_2-auc:0.8

69

### XGB approx

In [6]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'approx'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.741489	validation_1-auc:0.694397	validation_2-auc:0.634411
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.810935	validation_1-auc:0.746059	validation_2-auc:0.695953
[2]	validation_0-auc:0.837625	validation_1-auc:0.758212	validation_2-auc:0.725112
[3]	validation_0-auc:0.853271	validation_1-auc:0.761268	validation_2-auc:0.736353
[4]	validation_0-auc:0.872288	validation_1-auc:0.782171	validation_2-auc:0.760655
[5]	validation_0-auc:0.883565	validation_1-auc:0.793322	validation_2-auc:0.781332
[6]	validation_0-auc:0.891721	validation_1-auc:0.798243	validation_2-auc:0.792341
[7]	validation_0-auc:0.899822	validation_1-auc:0.808176	validation_2-auc:0.797289
[8]	validation_0-auc:0.904289	validation_1-auc:0.813067	validation_2-auc:0.80128
[9]	validation_0-auc:0.904749	validation_1-auc:0.817019	validation_2-auc:0.801411
[10]	validation_0-auc:0.

[98]	validation_0-auc:0.993654	validation_1-auc:0.890057	validation_2-auc:0.887105
[99]	validation_0-auc:0.994058	validation_1-auc:0.890497	validation_2-auc:0.88682
[100]	validation_0-auc:0.994241	validation_1-auc:0.890506	validation_2-auc:0.887766
[101]	validation_0-auc:0.994356	validation_1-auc:0.890552	validation_2-auc:0.888268
[102]	validation_0-auc:0.994488	validation_1-auc:0.890929	validation_2-auc:0.888625
[103]	validation_0-auc:0.99473	validation_1-auc:0.891526	validation_2-auc:0.889103
[104]	validation_0-auc:0.995014	validation_1-auc:0.891159	validation_2-auc:0.889381
[105]	validation_0-auc:0.995207	validation_1-auc:0.891045	validation_2-auc:0.889361
[106]	validation_0-auc:0.995327	validation_1-auc:0.891494	validation_2-auc:0.889989
[107]	validation_0-auc:0.995546	validation_1-auc:0.891567	validation_2-auc:0.890824
[108]	validation_0-auc:0.995573	validation_1-auc:0.891657	validation_2-auc:0.890513
[109]	validation_0-auc:0.995704	validation_1-auc:0.892407	validation_2-auc:0.890

69

### XGB hist

In [7]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'hist'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.758678	validation_1-auc:0.701693	validation_2-auc:0.68721
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.830171	validation_1-auc:0.749277	validation_2-auc:0.739985
[2]	validation_0-auc:0.865308	validation_1-auc:0.789733	validation_2-auc:0.768419
[3]	validation_0-auc:0.883783	validation_1-auc:0.807673	validation_2-auc:0.778614
[4]	validation_0-auc:0.890558	validation_1-auc:0.811595	validation_2-auc:0.783401
[5]	validation_0-auc:0.898783	validation_1-auc:0.817967	validation_2-auc:0.788554
[6]	validation_0-auc:0.904863	validation_1-auc:0.822693	validation_2-auc:0.792735
[7]	validation_0-auc:0.908484	validation_1-auc:0.822294	validation_2-auc:0.797284
[8]	validation_0-auc:0.913347	validation_1-auc:0.822981	validation_2-auc:0.803481
[9]	validation_0-auc:0.919034	validation_1-auc:0.829818	validation_2-auc:0.808588
[10]	validation_0-auc:0.

69

In [8]:
del X_train, y_train, X_valid, y_valid, X_test, y_test
gc.collect()

0

## Medium dataset

In [9]:
X, y = make_classification(n_samples=100_000,
                           n_features=20,
                           n_informative=10,
                           n_redundant=5,
                           n_repeated=0,
                           n_classes=2,
                           n_clusters_per_class=4,
                           flip_y=0.03,
                           class_sep=0.5,
                           shuffle=True,
                           random_state=0)

In [10]:
# create a train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=7)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)
del X, y
gc.collect()

40

In [11]:
X_train.shape

(70000, 20)

### XGB exact

In [12]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'exact'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.778947	validation_1-auc:0.760336	validation_2-auc:0.752741
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.829118	validation_1-auc:0.808962	validation_2-auc:0.806304
[2]	validation_0-auc:0.838417	validation_1-auc:0.81945	validation_2-auc:0.815563
[3]	validation_0-auc:0.843764	validation_1-auc:0.823889	validation_2-auc:0.822013
[4]	validation_0-auc:0.85715	validation_1-auc:0.836633	validation_2-auc:0.835781
[5]	validation_0-auc:0.868203	validation_1-auc:0.846455	validation_2-auc:0.847035
[6]	validation_0-auc:0.87291	validation_1-auc:0.851414	validation_2-auc:0.850911
[7]	validation_0-auc:0.876614	validation_1-auc:0.854894	validation_2-auc:0.854318
[8]	validation_0-auc:0.88329	validation_1-auc:0.86265	validation_2-auc:0.861111
[9]	validation_0-auc:0.885835	validation_1-auc:0.865122	validation_2-auc:0.864032
[10]	validation_0-auc:0.8895

[98]	validation_0-auc:0.958138	validation_1-auc:0.933501	validation_2-auc:0.930677
[99]	validation_0-auc:0.958252	validation_1-auc:0.933574	validation_2-auc:0.930719
[100]	validation_0-auc:0.958563	validation_1-auc:0.933841	validation_2-auc:0.930898
[101]	validation_0-auc:0.958787	validation_1-auc:0.933946	validation_2-auc:0.931056
[102]	validation_0-auc:0.959071	validation_1-auc:0.93417	validation_2-auc:0.931311
[103]	validation_0-auc:0.959655	validation_1-auc:0.934751	validation_2-auc:0.931801
[104]	validation_0-auc:0.960174	validation_1-auc:0.935188	validation_2-auc:0.932193
[105]	validation_0-auc:0.960439	validation_1-auc:0.935361	validation_2-auc:0.932342
[106]	validation_0-auc:0.960536	validation_1-auc:0.935344	validation_2-auc:0.932336
[107]	validation_0-auc:0.960693	validation_1-auc:0.935524	validation_2-auc:0.932441
[108]	validation_0-auc:0.960739	validation_1-auc:0.935517	validation_2-auc:0.93247
[109]	validation_0-auc:0.961175	validation_1-auc:0.935895	validation_2-auc:0.932

[196]	validation_0-auc:0.974155	validation_1-auc:0.942666	validation_2-auc:0.94042
[197]	validation_0-auc:0.97452	validation_1-auc:0.942874	validation_2-auc:0.940735
[198]	validation_0-auc:0.974587	validation_1-auc:0.942861	validation_2-auc:0.940721
[199]	validation_0-auc:0.974671	validation_1-auc:0.942874	validation_2-auc:0.940769
[200]	validation_0-auc:0.974732	validation_1-auc:0.942849	validation_2-auc:0.94074
[201]	validation_0-auc:0.974773	validation_1-auc:0.942897	validation_2-auc:0.940755
[202]	validation_0-auc:0.974933	validation_1-auc:0.943055	validation_2-auc:0.940965
[203]	validation_0-auc:0.975137	validation_1-auc:0.943202	validation_2-auc:0.941106
[204]	validation_0-auc:0.975172	validation_1-auc:0.94319	validation_2-auc:0.941109
[205]	validation_0-auc:0.97532	validation_1-auc:0.94322	validation_2-auc:0.941161
[206]	validation_0-auc:0.975368	validation_1-auc:0.943284	validation_2-auc:0.9412
[207]	validation_0-auc:0.975395	validation_1-auc:0.9433	validation_2-auc:0.941221
[2

[294]	validation_0-auc:0.983301	validation_1-auc:0.947536	validation_2-auc:0.945793
[295]	validation_0-auc:0.98334	validation_1-auc:0.947528	validation_2-auc:0.94581
[296]	validation_0-auc:0.983401	validation_1-auc:0.947598	validation_2-auc:0.945815
[297]	validation_0-auc:0.983485	validation_1-auc:0.947622	validation_2-auc:0.945814
[298]	validation_0-auc:0.98351	validation_1-auc:0.947641	validation_2-auc:0.945807
[299]	validation_0-auc:0.983662	validation_1-auc:0.947715	validation_2-auc:0.94593
[300]	validation_0-auc:0.983687	validation_1-auc:0.947727	validation_2-auc:0.945932
[301]	validation_0-auc:0.983745	validation_1-auc:0.947723	validation_2-auc:0.945914
[302]	validation_0-auc:0.983789	validation_1-auc:0.947706	validation_2-auc:0.94591
[303]	validation_0-auc:0.983837	validation_1-auc:0.947704	validation_2-auc:0.945886
[304]	validation_0-auc:0.983929	validation_1-auc:0.947775	validation_2-auc:0.945901
[305]	validation_0-auc:0.983976	validation_1-auc:0.947784	validation_2-auc:0.9459

[392]	validation_0-auc:0.98956	validation_1-auc:0.950385	validation_2-auc:0.948699
[393]	validation_0-auc:0.989581	validation_1-auc:0.950364	validation_2-auc:0.948717
[394]	validation_0-auc:0.989644	validation_1-auc:0.950349	validation_2-auc:0.948668
[395]	validation_0-auc:0.989689	validation_1-auc:0.950337	validation_2-auc:0.948682
[396]	validation_0-auc:0.989777	validation_1-auc:0.950305	validation_2-auc:0.948667
[397]	validation_0-auc:0.989849	validation_1-auc:0.950338	validation_2-auc:0.948733
[398]	validation_0-auc:0.989892	validation_1-auc:0.950336	validation_2-auc:0.948744
[399]	validation_0-auc:0.98994	validation_1-auc:0.950352	validation_2-auc:0.948839
[400]	validation_0-auc:0.990006	validation_1-auc:0.950364	validation_2-auc:0.948844
[401]	validation_0-auc:0.990055	validation_1-auc:0.950395	validation_2-auc:0.948829
[402]	validation_0-auc:0.990094	validation_1-auc:0.950405	validation_2-auc:0.948875
[403]	validation_0-auc:0.990146	validation_1-auc:0.95039	validation_2-auc:0.94

[490]	validation_0-auc:0.993363	validation_1-auc:0.951605	validation_2-auc:0.950512
[491]	validation_0-auc:0.993386	validation_1-auc:0.951609	validation_2-auc:0.950537
[492]	validation_0-auc:0.99341	validation_1-auc:0.951613	validation_2-auc:0.950533
[493]	validation_0-auc:0.993429	validation_1-auc:0.951607	validation_2-auc:0.950529
[494]	validation_0-auc:0.993446	validation_1-auc:0.951595	validation_2-auc:0.950551
[495]	validation_0-auc:0.993482	validation_1-auc:0.951552	validation_2-auc:0.950568
[496]	validation_0-auc:0.993547	validation_1-auc:0.951611	validation_2-auc:0.950591
[497]	validation_0-auc:0.993591	validation_1-auc:0.95164	validation_2-auc:0.950594
[498]	validation_0-auc:0.993639	validation_1-auc:0.951706	validation_2-auc:0.950643
[499]	validation_0-auc:0.993694	validation_1-auc:0.95173	validation_2-auc:0.950638
[500]	validation_0-auc:0.993715	validation_1-auc:0.951716	validation_2-auc:0.950633
[501]	validation_0-auc:0.993752	validation_1-auc:0.951744	validation_2-auc:0.95

14

### XGB approx

In [13]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'approx'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.775976	validation_1-auc:0.758807	validation_2-auc:0.762013
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.801329	validation_1-auc:0.785407	validation_2-auc:0.784544
[2]	validation_0-auc:0.826655	validation_1-auc:0.81195	validation_2-auc:0.807996
[3]	validation_0-auc:0.844678	validation_1-auc:0.830453	validation_2-auc:0.825746
[4]	validation_0-auc:0.853938	validation_1-auc:0.839455	validation_2-auc:0.835278
[5]	validation_0-auc:0.861534	validation_1-auc:0.846822	validation_2-auc:0.843673
[6]	validation_0-auc:0.86609	validation_1-auc:0.850846	validation_2-auc:0.847745
[7]	validation_0-auc:0.871773	validation_1-auc:0.856157	validation_2-auc:0.852901
[8]	validation_0-auc:0.873621	validation_1-auc:0.857819	validation_2-auc:0.854424
[9]	validation_0-auc:0.879859	validation_1-auc:0.863861	validation_2-auc:0.860981
[10]	validation_0-auc:0.8

[98]	validation_0-auc:0.958966	validation_1-auc:0.936325	validation_2-auc:0.933793
[99]	validation_0-auc:0.959118	validation_1-auc:0.936281	validation_2-auc:0.933817
[100]	validation_0-auc:0.959317	validation_1-auc:0.936378	validation_2-auc:0.933889
[101]	validation_0-auc:0.959575	validation_1-auc:0.936627	validation_2-auc:0.934091
[102]	validation_0-auc:0.95978	validation_1-auc:0.936652	validation_2-auc:0.934051
[103]	validation_0-auc:0.960125	validation_1-auc:0.936769	validation_2-auc:0.934172
[104]	validation_0-auc:0.960559	validation_1-auc:0.937127	validation_2-auc:0.934582
[105]	validation_0-auc:0.960797	validation_1-auc:0.937281	validation_2-auc:0.934666
[106]	validation_0-auc:0.961103	validation_1-auc:0.937546	validation_2-auc:0.934968
[107]	validation_0-auc:0.961268	validation_1-auc:0.937588	validation_2-auc:0.935013
[108]	validation_0-auc:0.961485	validation_1-auc:0.937808	validation_2-auc:0.935297
[109]	validation_0-auc:0.961779	validation_1-auc:0.937972	validation_2-auc:0.93

[197]	validation_0-auc:0.979054	validation_1-auc:0.94868	validation_2-auc:0.947058
[198]	validation_0-auc:0.979167	validation_1-auc:0.9487	validation_2-auc:0.947088
[199]	validation_0-auc:0.979331	validation_1-auc:0.948849	validation_2-auc:0.947226
[200]	validation_0-auc:0.979448	validation_1-auc:0.948868	validation_2-auc:0.947209
[201]	validation_0-auc:0.979577	validation_1-auc:0.948973	validation_2-auc:0.947317
[202]	validation_0-auc:0.979691	validation_1-auc:0.949042	validation_2-auc:0.947358
[203]	validation_0-auc:0.979804	validation_1-auc:0.949139	validation_2-auc:0.947344
[204]	validation_0-auc:0.979933	validation_1-auc:0.949195	validation_2-auc:0.947402
[205]	validation_0-auc:0.980026	validation_1-auc:0.949208	validation_2-auc:0.947452
[206]	validation_0-auc:0.980152	validation_1-auc:0.949319	validation_2-auc:0.947611
[207]	validation_0-auc:0.98026	validation_1-auc:0.949377	validation_2-auc:0.947667
[208]	validation_0-auc:0.980437	validation_1-auc:0.949484	validation_2-auc:0.947

[295]	validation_0-auc:0.989137	validation_1-auc:0.952921	validation_2-auc:0.951611
[296]	validation_0-auc:0.989194	validation_1-auc:0.952946	validation_2-auc:0.951622
[297]	validation_0-auc:0.98927	validation_1-auc:0.953016	validation_2-auc:0.951667
[298]	validation_0-auc:0.989327	validation_1-auc:0.953052	validation_2-auc:0.951778
[299]	validation_0-auc:0.989392	validation_1-auc:0.953098	validation_2-auc:0.951764
[300]	validation_0-auc:0.989438	validation_1-auc:0.953098	validation_2-auc:0.951766
[301]	validation_0-auc:0.989536	validation_1-auc:0.953186	validation_2-auc:0.951861
[302]	validation_0-auc:0.989597	validation_1-auc:0.9532	validation_2-auc:0.95189
[303]	validation_0-auc:0.989631	validation_1-auc:0.953228	validation_2-auc:0.951894
[304]	validation_0-auc:0.989727	validation_1-auc:0.953302	validation_2-auc:0.951988
[305]	validation_0-auc:0.989806	validation_1-auc:0.953373	validation_2-auc:0.952013
[306]	validation_0-auc:0.989844	validation_1-auc:0.953357	validation_2-auc:0.952

[393]	validation_0-auc:0.994468	validation_1-auc:0.955195	validation_2-auc:0.953707
[394]	validation_0-auc:0.994501	validation_1-auc:0.955211	validation_2-auc:0.953756
[395]	validation_0-auc:0.99453	validation_1-auc:0.955199	validation_2-auc:0.953777
[396]	validation_0-auc:0.994566	validation_1-auc:0.955217	validation_2-auc:0.953743
[397]	validation_0-auc:0.99459	validation_1-auc:0.955238	validation_2-auc:0.953751
[398]	validation_0-auc:0.994629	validation_1-auc:0.955253	validation_2-auc:0.953782
[399]	validation_0-auc:0.994668	validation_1-auc:0.955255	validation_2-auc:0.953835
[400]	validation_0-auc:0.994716	validation_1-auc:0.955269	validation_2-auc:0.953817
[401]	validation_0-auc:0.994763	validation_1-auc:0.955228	validation_2-auc:0.9538
[402]	validation_0-auc:0.994813	validation_1-auc:0.955241	validation_2-auc:0.953817
[403]	validation_0-auc:0.994838	validation_1-auc:0.955271	validation_2-auc:0.95382
[404]	validation_0-auc:0.994882	validation_1-auc:0.955267	validation_2-auc:0.9538

69

### XGB hist

In [14]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'hist'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.77806	validation_1-auc:0.762036	validation_2-auc:0.753627
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.818108	validation_1-auc:0.798077	validation_2-auc:0.794009
[2]	validation_0-auc:0.839773	validation_1-auc:0.822802	validation_2-auc:0.815827
[3]	validation_0-auc:0.849957	validation_1-auc:0.831753	validation_2-auc:0.8237
[4]	validation_0-auc:0.86041	validation_1-auc:0.844101	validation_2-auc:0.835701
[5]	validation_0-auc:0.864665	validation_1-auc:0.848513	validation_2-auc:0.840165
[6]	validation_0-auc:0.869621	validation_1-auc:0.853409	validation_2-auc:0.844468
[7]	validation_0-auc:0.877695	validation_1-auc:0.861245	validation_2-auc:0.853271
[8]	validation_0-auc:0.88225	validation_1-auc:0.865373	validation_2-auc:0.858509
[9]	validation_0-auc:0.886391	validation_1-auc:0.868513	validation_2-auc:0.862728
[10]	validation_0-auc:0.8887

[98]	validation_0-auc:0.957399	validation_1-auc:0.932746	validation_2-auc:0.930045
[99]	validation_0-auc:0.957687	validation_1-auc:0.932894	validation_2-auc:0.930173
[100]	validation_0-auc:0.957998	validation_1-auc:0.933121	validation_2-auc:0.930398
[101]	validation_0-auc:0.958136	validation_1-auc:0.933132	validation_2-auc:0.93044
[102]	validation_0-auc:0.958339	validation_1-auc:0.933233	validation_2-auc:0.930499
[103]	validation_0-auc:0.958391	validation_1-auc:0.933244	validation_2-auc:0.930518
[104]	validation_0-auc:0.958632	validation_1-auc:0.933501	validation_2-auc:0.930789
[105]	validation_0-auc:0.958858	validation_1-auc:0.933721	validation_2-auc:0.930985
[106]	validation_0-auc:0.958959	validation_1-auc:0.93373	validation_2-auc:0.931016
[107]	validation_0-auc:0.959231	validation_1-auc:0.933999	validation_2-auc:0.93132
[108]	validation_0-auc:0.959375	validation_1-auc:0.93407	validation_2-auc:0.931355
[109]	validation_0-auc:0.959735	validation_1-auc:0.934404	validation_2-auc:0.93173

[196]	validation_0-auc:0.976101	validation_1-auc:0.944079	validation_2-auc:0.943178
[197]	validation_0-auc:0.976243	validation_1-auc:0.944037	validation_2-auc:0.943203
[198]	validation_0-auc:0.97639	validation_1-auc:0.944144	validation_2-auc:0.943409
[199]	validation_0-auc:0.976469	validation_1-auc:0.944203	validation_2-auc:0.943503
[200]	validation_0-auc:0.976617	validation_1-auc:0.944333	validation_2-auc:0.943596
[201]	validation_0-auc:0.976739	validation_1-auc:0.944372	validation_2-auc:0.943659
[202]	validation_0-auc:0.976914	validation_1-auc:0.944555	validation_2-auc:0.943793
[203]	validation_0-auc:0.977092	validation_1-auc:0.944663	validation_2-auc:0.943916
[204]	validation_0-auc:0.977144	validation_1-auc:0.944654	validation_2-auc:0.943902
[205]	validation_0-auc:0.977245	validation_1-auc:0.944647	validation_2-auc:0.943907
[206]	validation_0-auc:0.977314	validation_1-auc:0.944646	validation_2-auc:0.943964
[207]	validation_0-auc:0.977552	validation_1-auc:0.944895	validation_2-auc:0.

[294]	validation_0-auc:0.986521	validation_1-auc:0.949751	validation_2-auc:0.94926
[295]	validation_0-auc:0.986548	validation_1-auc:0.949764	validation_2-auc:0.949262
[296]	validation_0-auc:0.986651	validation_1-auc:0.949816	validation_2-auc:0.949361
[297]	validation_0-auc:0.986763	validation_1-auc:0.949906	validation_2-auc:0.949364
[298]	validation_0-auc:0.986801	validation_1-auc:0.94991	validation_2-auc:0.949401
[299]	validation_0-auc:0.986896	validation_1-auc:0.949925	validation_2-auc:0.949385
[300]	validation_0-auc:0.986987	validation_1-auc:0.950018	validation_2-auc:0.949456
[301]	validation_0-auc:0.987065	validation_1-auc:0.950059	validation_2-auc:0.949512
[302]	validation_0-auc:0.987123	validation_1-auc:0.950019	validation_2-auc:0.949487
[303]	validation_0-auc:0.987155	validation_1-auc:0.950018	validation_2-auc:0.949493
[304]	validation_0-auc:0.987227	validation_1-auc:0.950048	validation_2-auc:0.949509
[305]	validation_0-auc:0.987352	validation_1-auc:0.950178	validation_2-auc:0.9

[392]	validation_0-auc:0.992524	validation_1-auc:0.952344	validation_2-auc:0.951775
[393]	validation_0-auc:0.992578	validation_1-auc:0.952362	validation_2-auc:0.95182
[394]	validation_0-auc:0.992597	validation_1-auc:0.952386	validation_2-auc:0.951837
[395]	validation_0-auc:0.992647	validation_1-auc:0.952429	validation_2-auc:0.951892
[396]	validation_0-auc:0.992698	validation_1-auc:0.952428	validation_2-auc:0.951891
[397]	validation_0-auc:0.992741	validation_1-auc:0.95242	validation_2-auc:0.951903
[398]	validation_0-auc:0.99277	validation_1-auc:0.952421	validation_2-auc:0.951878
[399]	validation_0-auc:0.99283	validation_1-auc:0.952422	validation_2-auc:0.95194
[400]	validation_0-auc:0.992882	validation_1-auc:0.952446	validation_2-auc:0.951966
[401]	validation_0-auc:0.99292	validation_1-auc:0.952452	validation_2-auc:0.951944
[402]	validation_0-auc:0.992965	validation_1-auc:0.952426	validation_2-auc:0.951966
[403]	validation_0-auc:0.993028	validation_1-auc:0.952455	validation_2-auc:0.95202

69

In [15]:
del X_train, y_train, X_valid, y_valid, X_test, y_test
gc.collect()

0

## Medium dataset large col

In [16]:
X, y = make_classification(n_samples=100_000,
                           n_features=500,
                           n_informative=300,
                           n_redundant=100,
                           n_repeated=50,
                           n_classes=2,
                           n_clusters_per_class=2,
                           flip_y=0.03,
                           class_sep=0.5,
                           shuffle=True,
                           random_state=0)

In [17]:
# create a train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=7)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)
del X, y
gc.collect()

40

In [18]:
X_train.shape

(70000, 500)

### XGB exact

In [19]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'exact'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.646665	validation_1-auc:0.612675	validation_2-auc:0.613825
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.7127	validation_1-auc:0.657941	validation_2-auc:0.656126
[2]	validation_0-auc:0.757517	validation_1-auc:0.692135	validation_2-auc:0.692438
[3]	validation_0-auc:0.785799	validation_1-auc:0.70974	validation_2-auc:0.711287
[4]	validation_0-auc:0.810365	validation_1-auc:0.730969	validation_2-auc:0.733148
[5]	validation_0-auc:0.827708	validation_1-auc:0.745059	validation_2-auc:0.747095
[6]	validation_0-auc:0.848045	validation_1-auc:0.763189	validation_2-auc:0.763066
[7]	validation_0-auc:0.858984	validation_1-auc:0.774676	validation_2-auc:0.77342
[8]	validation_0-auc:0.870722	validation_1-auc:0.78483	validation_2-auc:0.783666
[9]	validation_0-auc:0.879865	validation_1-auc:0.792036	validation_2-auc:0.793223
[10]	validation_0-auc:0.8926

[98]	validation_0-auc:0.996149	validation_1-auc:0.959473	validation_2-auc:0.960251
[99]	validation_0-auc:0.996193	validation_1-auc:0.959725	validation_2-auc:0.960537
[100]	validation_0-auc:0.996272	validation_1-auc:0.959952	validation_2-auc:0.960886
[101]	validation_0-auc:0.996322	validation_1-auc:0.960086	validation_2-auc:0.960995
[102]	validation_0-auc:0.996414	validation_1-auc:0.960472	validation_2-auc:0.961443
[103]	validation_0-auc:0.996481	validation_1-auc:0.960759	validation_2-auc:0.961823
[104]	validation_0-auc:0.996548	validation_1-auc:0.960996	validation_2-auc:0.961982
[105]	validation_0-auc:0.996611	validation_1-auc:0.961124	validation_2-auc:0.962151
[106]	validation_0-auc:0.99669	validation_1-auc:0.961468	validation_2-auc:0.962498
[107]	validation_0-auc:0.996746	validation_1-auc:0.961625	validation_2-auc:0.962645
[108]	validation_0-auc:0.996798	validation_1-auc:0.961786	validation_2-auc:0.962928
[109]	validation_0-auc:0.996838	validation_1-auc:0.961883	validation_2-auc:0.96

[196]	validation_0-auc:0.999367	validation_1-auc:0.974481	validation_2-auc:0.975561
[197]	validation_0-auc:0.999384	validation_1-auc:0.974627	validation_2-auc:0.975618
[198]	validation_0-auc:0.999393	validation_1-auc:0.974673	validation_2-auc:0.975706
[199]	validation_0-auc:0.999398	validation_1-auc:0.974696	validation_2-auc:0.975774
[200]	validation_0-auc:0.999419	validation_1-auc:0.974847	validation_2-auc:0.976011
[201]	validation_0-auc:0.999425	validation_1-auc:0.974843	validation_2-auc:0.976029
[202]	validation_0-auc:0.99943	validation_1-auc:0.974924	validation_2-auc:0.976174
[203]	validation_0-auc:0.999435	validation_1-auc:0.974992	validation_2-auc:0.976223
[204]	validation_0-auc:0.999448	validation_1-auc:0.975028	validation_2-auc:0.976319
[205]	validation_0-auc:0.999461	validation_1-auc:0.975205	validation_2-auc:0.976386
[206]	validation_0-auc:0.999476	validation_1-auc:0.975235	validation_2-auc:0.976527
[207]	validation_0-auc:0.999483	validation_1-auc:0.975249	validation_2-auc:0.

[294]	validation_0-auc:0.999893	validation_1-auc:0.978794	validation_2-auc:0.980109
[295]	validation_0-auc:0.999895	validation_1-auc:0.978841	validation_2-auc:0.980105
[296]	validation_0-auc:0.999898	validation_1-auc:0.978864	validation_2-auc:0.98008
[297]	validation_0-auc:0.9999	validation_1-auc:0.978884	validation_2-auc:0.980122
[298]	validation_0-auc:0.999902	validation_1-auc:0.978937	validation_2-auc:0.980126
[299]	validation_0-auc:0.999904	validation_1-auc:0.978974	validation_2-auc:0.980134
[300]	validation_0-auc:0.999907	validation_1-auc:0.979014	validation_2-auc:0.98018
[301]	validation_0-auc:0.999908	validation_1-auc:0.97905	validation_2-auc:0.980204
[302]	validation_0-auc:0.99991	validation_1-auc:0.979034	validation_2-auc:0.980233
[303]	validation_0-auc:0.999913	validation_1-auc:0.979018	validation_2-auc:0.980249
[304]	validation_0-auc:0.999916	validation_1-auc:0.979026	validation_2-auc:0.980302
[305]	validation_0-auc:0.999917	validation_1-auc:0.979034	validation_2-auc:0.98031

[392]	validation_0-auc:0.999985	validation_1-auc:0.980652	validation_2-auc:0.98201
[393]	validation_0-auc:0.999986	validation_1-auc:0.980716	validation_2-auc:0.982042
[394]	validation_0-auc:0.999986	validation_1-auc:0.980747	validation_2-auc:0.982076
[395]	validation_0-auc:0.999986	validation_1-auc:0.98078	validation_2-auc:0.982116
[396]	validation_0-auc:0.999987	validation_1-auc:0.980761	validation_2-auc:0.98212
[397]	validation_0-auc:0.999987	validation_1-auc:0.980784	validation_2-auc:0.98216
[398]	validation_0-auc:0.999987	validation_1-auc:0.980802	validation_2-auc:0.982181
[399]	validation_0-auc:0.999988	validation_1-auc:0.980805	validation_2-auc:0.982208
[400]	validation_0-auc:0.999988	validation_1-auc:0.980817	validation_2-auc:0.982239
[401]	validation_0-auc:0.999988	validation_1-auc:0.980837	validation_2-auc:0.982273
[402]	validation_0-auc:0.999989	validation_1-auc:0.980884	validation_2-auc:0.982271
[403]	validation_0-auc:0.999989	validation_1-auc:0.980931	validation_2-auc:0.982

[490]	validation_0-auc:0.999999	validation_1-auc:0.982177	validation_2-auc:0.983448
[491]	validation_0-auc:0.999999	validation_1-auc:0.982202	validation_2-auc:0.983449
[492]	validation_0-auc:0.999999	validation_1-auc:0.982197	validation_2-auc:0.983442
[493]	validation_0-auc:0.999999	validation_1-auc:0.982208	validation_2-auc:0.983451
Stopping. Best iteration:
[483]	validation_0-auc:0.999999	validation_1-auc:0.982118	validation_2-auc:0.983465

CPU times: user 1h 17min 9s, sys: 6.43 s, total: 1h 17min 16s
Wall time: 10min 33s


14

### XGB approx

In [20]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'approx'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.659636	validation_1-auc:0.617572	validation_2-auc:0.611425
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.712476	validation_1-auc:0.65664	validation_2-auc:0.65297
[2]	validation_0-auc:0.755993	validation_1-auc:0.689511	validation_2-auc:0.688692
[3]	validation_0-auc:0.781053	validation_1-auc:0.710915	validation_2-auc:0.707682
[4]	validation_0-auc:0.803935	validation_1-auc:0.728925	validation_2-auc:0.729111
[5]	validation_0-auc:0.828587	validation_1-auc:0.74924	validation_2-auc:0.751246
[6]	validation_0-auc:0.842206	validation_1-auc:0.762542	validation_2-auc:0.760858
[7]	validation_0-auc:0.856427	validation_1-auc:0.775716	validation_2-auc:0.775057
[8]	validation_0-auc:0.86987	validation_1-auc:0.787558	validation_2-auc:0.78885
[9]	validation_0-auc:0.880896	validation_1-auc:0.798046	validation_2-auc:0.799734
[10]	validation_0-auc:0.8890

[98]	validation_0-auc:0.995618	validation_1-auc:0.961505	validation_2-auc:0.963399
[99]	validation_0-auc:0.995717	validation_1-auc:0.961654	validation_2-auc:0.963797
[100]	validation_0-auc:0.995833	validation_1-auc:0.962043	validation_2-auc:0.96423
[101]	validation_0-auc:0.995911	validation_1-auc:0.962333	validation_2-auc:0.964525
[102]	validation_0-auc:0.995959	validation_1-auc:0.962617	validation_2-auc:0.964918
[103]	validation_0-auc:0.996026	validation_1-auc:0.96286	validation_2-auc:0.965099
[104]	validation_0-auc:0.996112	validation_1-auc:0.963161	validation_2-auc:0.965409
[105]	validation_0-auc:0.996186	validation_1-auc:0.963341	validation_2-auc:0.965585
[106]	validation_0-auc:0.996248	validation_1-auc:0.963605	validation_2-auc:0.965845
[107]	validation_0-auc:0.996303	validation_1-auc:0.963877	validation_2-auc:0.966114
[108]	validation_0-auc:0.996356	validation_1-auc:0.964024	validation_2-auc:0.96635
[109]	validation_0-auc:0.996428	validation_1-auc:0.964336	validation_2-auc:0.9666

[196]	validation_0-auc:0.999395	validation_1-auc:0.976	validation_2-auc:0.977506
[197]	validation_0-auc:0.999406	validation_1-auc:0.976048	validation_2-auc:0.977552
[198]	validation_0-auc:0.999413	validation_1-auc:0.976093	validation_2-auc:0.977634
[199]	validation_0-auc:0.999421	validation_1-auc:0.976131	validation_2-auc:0.97765
[200]	validation_0-auc:0.99943	validation_1-auc:0.976215	validation_2-auc:0.977706
[201]	validation_0-auc:0.999441	validation_1-auc:0.97627	validation_2-auc:0.977777
[202]	validation_0-auc:0.99945	validation_1-auc:0.976359	validation_2-auc:0.977827
[203]	validation_0-auc:0.999462	validation_1-auc:0.976344	validation_2-auc:0.977811
[204]	validation_0-auc:0.999475	validation_1-auc:0.976393	validation_2-auc:0.977886
[205]	validation_0-auc:0.999486	validation_1-auc:0.97648	validation_2-auc:0.977947
[206]	validation_0-auc:0.9995	validation_1-auc:0.976522	validation_2-auc:0.977969
[207]	validation_0-auc:0.999514	validation_1-auc:0.976578	validation_2-auc:0.977978
[2

[294]	validation_0-auc:0.999931	validation_1-auc:0.980458	validation_2-auc:0.981263
[295]	validation_0-auc:0.999932	validation_1-auc:0.980473	validation_2-auc:0.98129
[296]	validation_0-auc:0.999933	validation_1-auc:0.980447	validation_2-auc:0.981308
[297]	validation_0-auc:0.999934	validation_1-auc:0.980466	validation_2-auc:0.981358
[298]	validation_0-auc:0.999936	validation_1-auc:0.980487	validation_2-auc:0.981404
[299]	validation_0-auc:0.999937	validation_1-auc:0.980484	validation_2-auc:0.981414
[300]	validation_0-auc:0.999938	validation_1-auc:0.980476	validation_2-auc:0.981447
[301]	validation_0-auc:0.99994	validation_1-auc:0.980504	validation_2-auc:0.981467
[302]	validation_0-auc:0.999941	validation_1-auc:0.980499	validation_2-auc:0.981467
[303]	validation_0-auc:0.999943	validation_1-auc:0.980557	validation_2-auc:0.981491
[304]	validation_0-auc:0.999944	validation_1-auc:0.980569	validation_2-auc:0.981486
[305]	validation_0-auc:0.999945	validation_1-auc:0.980625	validation_2-auc:0.9

[392]	validation_0-auc:0.999996	validation_1-auc:0.982379	validation_2-auc:0.983145
[393]	validation_0-auc:0.999996	validation_1-auc:0.982367	validation_2-auc:0.983128
[394]	validation_0-auc:0.999996	validation_1-auc:0.98239	validation_2-auc:0.983169
[395]	validation_0-auc:0.999996	validation_1-auc:0.982421	validation_2-auc:0.983201
[396]	validation_0-auc:0.999996	validation_1-auc:0.982443	validation_2-auc:0.983189
[397]	validation_0-auc:0.999997	validation_1-auc:0.982467	validation_2-auc:0.983231
[398]	validation_0-auc:0.999997	validation_1-auc:0.982486	validation_2-auc:0.983259
[399]	validation_0-auc:0.999997	validation_1-auc:0.982489	validation_2-auc:0.983287
[400]	validation_0-auc:0.999997	validation_1-auc:0.982487	validation_2-auc:0.983285
[401]	validation_0-auc:0.999997	validation_1-auc:0.982495	validation_2-auc:0.983301
[402]	validation_0-auc:0.999997	validation_1-auc:0.98249	validation_2-auc:0.983326
[403]	validation_0-auc:0.999997	validation_1-auc:0.982486	validation_2-auc:0.9

[495]	validation_0-auc:1	validation_1-auc:0.983139	validation_2-auc:0.984037
[496]	validation_0-auc:1	validation_1-auc:0.983146	validation_2-auc:0.984041
[497]	validation_0-auc:1	validation_1-auc:0.983172	validation_2-auc:0.984042
[498]	validation_0-auc:1	validation_1-auc:0.983221	validation_2-auc:0.984064
[499]	validation_0-auc:1	validation_1-auc:0.983224	validation_2-auc:0.984065
[500]	validation_0-auc:1	validation_1-auc:0.983235	validation_2-auc:0.984077
[501]	validation_0-auc:1	validation_1-auc:0.983229	validation_2-auc:0.984082
[502]	validation_0-auc:1	validation_1-auc:0.983242	validation_2-auc:0.984066
[503]	validation_0-auc:1	validation_1-auc:0.983247	validation_2-auc:0.984074
[504]	validation_0-auc:1	validation_1-auc:0.983241	validation_2-auc:0.984089
[505]	validation_0-auc:1	validation_1-auc:0.983268	validation_2-auc:0.98411
[506]	validation_0-auc:1	validation_1-auc:0.98328	validation_2-auc:0.98412
[507]	validation_0-auc:1	validation_1-auc:0.983294	validation_2-auc:0.984108
[5

69

### XGB hist

In [21]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'hist'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.654034	validation_1-auc:0.610072	validation_2-auc:0.614858
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.715064	validation_1-auc:0.654502	validation_2-auc:0.651426
[2]	validation_0-auc:0.758067	validation_1-auc:0.690625	validation_2-auc:0.679448
[3]	validation_0-auc:0.787825	validation_1-auc:0.715323	validation_2-auc:0.704908
[4]	validation_0-auc:0.813669	validation_1-auc:0.736025	validation_2-auc:0.726983
[5]	validation_0-auc:0.827657	validation_1-auc:0.749097	validation_2-auc:0.73857
[6]	validation_0-auc:0.84382	validation_1-auc:0.762815	validation_2-auc:0.756144
[7]	validation_0-auc:0.85498	validation_1-auc:0.771808	validation_2-auc:0.76821
[8]	validation_0-auc:0.865253	validation_1-auc:0.781789	validation_2-auc:0.775535
[9]	validation_0-auc:0.87846	validation_1-auc:0.792676	validation_2-auc:0.789724
[10]	validation_0-auc:0.8875

[98]	validation_0-auc:0.996232	validation_1-auc:0.962176	validation_2-auc:0.963639
[99]	validation_0-auc:0.996294	validation_1-auc:0.962566	validation_2-auc:0.964059
[100]	validation_0-auc:0.996416	validation_1-auc:0.963057	validation_2-auc:0.964519
[101]	validation_0-auc:0.996496	validation_1-auc:0.963429	validation_2-auc:0.964938
[102]	validation_0-auc:0.996557	validation_1-auc:0.963595	validation_2-auc:0.965087
[103]	validation_0-auc:0.996621	validation_1-auc:0.964001	validation_2-auc:0.965434
[104]	validation_0-auc:0.996676	validation_1-auc:0.964268	validation_2-auc:0.965644
[105]	validation_0-auc:0.996737	validation_1-auc:0.964468	validation_2-auc:0.965838
[106]	validation_0-auc:0.996792	validation_1-auc:0.964624	validation_2-auc:0.966171
[107]	validation_0-auc:0.996847	validation_1-auc:0.964877	validation_2-auc:0.966532
[108]	validation_0-auc:0.996907	validation_1-auc:0.965024	validation_2-auc:0.966499
[109]	validation_0-auc:0.996987	validation_1-auc:0.965258	validation_2-auc:0.9

[196]	validation_0-auc:0.999454	validation_1-auc:0.976607	validation_2-auc:0.978104
[197]	validation_0-auc:0.999458	validation_1-auc:0.976718	validation_2-auc:0.978098
[198]	validation_0-auc:0.999472	validation_1-auc:0.976803	validation_2-auc:0.978189
[199]	validation_0-auc:0.99948	validation_1-auc:0.976893	validation_2-auc:0.978249
[200]	validation_0-auc:0.999489	validation_1-auc:0.976953	validation_2-auc:0.978303
[201]	validation_0-auc:0.999501	validation_1-auc:0.976987	validation_2-auc:0.978423
[202]	validation_0-auc:0.999512	validation_1-auc:0.97708	validation_2-auc:0.97847
[203]	validation_0-auc:0.99952	validation_1-auc:0.977107	validation_2-auc:0.978527
[204]	validation_0-auc:0.999527	validation_1-auc:0.977097	validation_2-auc:0.978569
[205]	validation_0-auc:0.999539	validation_1-auc:0.977158	validation_2-auc:0.978632
[206]	validation_0-auc:0.999544	validation_1-auc:0.977162	validation_2-auc:0.978612
[207]	validation_0-auc:0.999552	validation_1-auc:0.977256	validation_2-auc:0.978

[294]	validation_0-auc:0.999928	validation_1-auc:0.980772	validation_2-auc:0.982194
[295]	validation_0-auc:0.999929	validation_1-auc:0.980782	validation_2-auc:0.982201
[296]	validation_0-auc:0.999931	validation_1-auc:0.980775	validation_2-auc:0.98219
[297]	validation_0-auc:0.999931	validation_1-auc:0.980778	validation_2-auc:0.982174
[298]	validation_0-auc:0.999933	validation_1-auc:0.980801	validation_2-auc:0.982247
[299]	validation_0-auc:0.999935	validation_1-auc:0.980813	validation_2-auc:0.98231
[300]	validation_0-auc:0.999936	validation_1-auc:0.980809	validation_2-auc:0.98234
[301]	validation_0-auc:0.999937	validation_1-auc:0.980841	validation_2-auc:0.982377
[302]	validation_0-auc:0.999939	validation_1-auc:0.980869	validation_2-auc:0.982408
[303]	validation_0-auc:0.99994	validation_1-auc:0.980881	validation_2-auc:0.982368
[304]	validation_0-auc:0.99994	validation_1-auc:0.980898	validation_2-auc:0.982388
[305]	validation_0-auc:0.999942	validation_1-auc:0.980926	validation_2-auc:0.9824

[393]	validation_0-auc:0.999993	validation_1-auc:0.982628	validation_2-auc:0.983689
[394]	validation_0-auc:0.999993	validation_1-auc:0.982629	validation_2-auc:0.983689
[395]	validation_0-auc:0.999993	validation_1-auc:0.982644	validation_2-auc:0.983676
[396]	validation_0-auc:0.999993	validation_1-auc:0.982648	validation_2-auc:0.983684
[397]	validation_0-auc:0.999994	validation_1-auc:0.982651	validation_2-auc:0.9837
[398]	validation_0-auc:0.999994	validation_1-auc:0.982655	validation_2-auc:0.98373
[399]	validation_0-auc:0.999994	validation_1-auc:0.982731	validation_2-auc:0.983715
[400]	validation_0-auc:0.999994	validation_1-auc:0.982715	validation_2-auc:0.983687
[401]	validation_0-auc:0.999994	validation_1-auc:0.982703	validation_2-auc:0.983689
[402]	validation_0-auc:0.999994	validation_1-auc:0.982701	validation_2-auc:0.9837
[403]	validation_0-auc:0.999994	validation_1-auc:0.982719	validation_2-auc:0.983722
[404]	validation_0-auc:0.999995	validation_1-auc:0.982741	validation_2-auc:0.9837

[492]	validation_0-auc:1	validation_1-auc:0.983594	validation_2-auc:0.984579
[493]	validation_0-auc:1	validation_1-auc:0.983602	validation_2-auc:0.984584
[494]	validation_0-auc:1	validation_1-auc:0.983612	validation_2-auc:0.984569
[495]	validation_0-auc:1	validation_1-auc:0.983601	validation_2-auc:0.984566
[496]	validation_0-auc:1	validation_1-auc:0.983618	validation_2-auc:0.984573
[497]	validation_0-auc:1	validation_1-auc:0.983609	validation_2-auc:0.984572
[498]	validation_0-auc:1	validation_1-auc:0.983614	validation_2-auc:0.984575
[499]	validation_0-auc:1	validation_1-auc:0.983618	validation_2-auc:0.984571
[500]	validation_0-auc:1	validation_1-auc:0.983603	validation_2-auc:0.984568
[501]	validation_0-auc:1	validation_1-auc:0.983622	validation_2-auc:0.984576
[502]	validation_0-auc:1	validation_1-auc:0.983623	validation_2-auc:0.984583
[503]	validation_0-auc:1	validation_1-auc:0.983614	validation_2-auc:0.984584
Stopping. Best iteration:
[493]	validation_0-auc:1	validation_1-auc:0.983602

69

In [22]:
del X_train, y_train, X_valid, y_valid, X_test, y_test
gc.collect()

0

## Huge dataset

In [23]:
X, y = make_classification(n_samples=1_000_000,
                           n_features=20,
                           n_informative=10,
                           n_redundant=5,
                           n_repeated=0,
                           n_classes=2,
                           n_clusters_per_class=4,
                           flip_y=0.03,
                           class_sep=0.5,
                           shuffle=True,
                           random_state=0)

In [24]:
# create a train/test split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=7)
X_test, X_valid, y_test, y_valid = train_test_split(X_valid, y_valid, test_size=0.5, random_state=7)
del X, y
gc.collect()

40

In [25]:
X_train.shape

(700000, 20)

### XGB exact

In [26]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'exact'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.78312	validation_1-auc:0.780204	validation_2-auc:0.779002
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.814396	validation_1-auc:0.811988	validation_2-auc:0.809615
[2]	validation_0-auc:0.825193	validation_1-auc:0.822523	validation_2-auc:0.820197
[3]	validation_0-auc:0.83865	validation_1-auc:0.83564	validation_2-auc:0.833542
[4]	validation_0-auc:0.847952	validation_1-auc:0.844743	validation_2-auc:0.843127
[5]	validation_0-auc:0.853264	validation_1-auc:0.849842	validation_2-auc:0.848196
[6]	validation_0-auc:0.85446	validation_1-auc:0.85132	validation_2-auc:0.84989
[7]	validation_0-auc:0.855344	validation_1-auc:0.85246	validation_2-auc:0.850841
[8]	validation_0-auc:0.85945	validation_1-auc:0.85647	validation_2-auc:0.855117
[9]	validation_0-auc:0.860249	validation_1-auc:0.857247	validation_2-auc:0.856056
[10]	validation_0-auc:0.861443	v

[98]	validation_0-auc:0.92937	validation_1-auc:0.923631	validation_2-auc:0.924426
[99]	validation_0-auc:0.929428	validation_1-auc:0.923678	validation_2-auc:0.924481
[100]	validation_0-auc:0.929461	validation_1-auc:0.9237	validation_2-auc:0.924502
[101]	validation_0-auc:0.929509	validation_1-auc:0.923746	validation_2-auc:0.924548
[102]	validation_0-auc:0.929613	validation_1-auc:0.923822	validation_2-auc:0.924617
[103]	validation_0-auc:0.929681	validation_1-auc:0.923879	validation_2-auc:0.924672
[104]	validation_0-auc:0.929931	validation_1-auc:0.92413	validation_2-auc:0.924894
[105]	validation_0-auc:0.930508	validation_1-auc:0.924708	validation_2-auc:0.925477
[106]	validation_0-auc:0.930546	validation_1-auc:0.924728	validation_2-auc:0.925512
[107]	validation_0-auc:0.930819	validation_1-auc:0.924998	validation_2-auc:0.925755
[108]	validation_0-auc:0.93109	validation_1-auc:0.925296	validation_2-auc:0.926056
[109]	validation_0-auc:0.931422	validation_1-auc:0.925633	validation_2-auc:0.926432

[196]	validation_0-auc:0.945512	validation_1-auc:0.938411	validation_2-auc:0.939428
[197]	validation_0-auc:0.945557	validation_1-auc:0.938437	validation_2-auc:0.939457
[198]	validation_0-auc:0.945625	validation_1-auc:0.938487	validation_2-auc:0.939506
[199]	validation_0-auc:0.945634	validation_1-auc:0.938492	validation_2-auc:0.93951
[200]	validation_0-auc:0.945857	validation_1-auc:0.938707	validation_2-auc:0.939734
[201]	validation_0-auc:0.945884	validation_1-auc:0.938715	validation_2-auc:0.93974
[202]	validation_0-auc:0.945956	validation_1-auc:0.938766	validation_2-auc:0.939781
[203]	validation_0-auc:0.94601	validation_1-auc:0.938802	validation_2-auc:0.939818
[204]	validation_0-auc:0.946234	validation_1-auc:0.939	validation_2-auc:0.940031
[205]	validation_0-auc:0.946272	validation_1-auc:0.939028	validation_2-auc:0.940045
[206]	validation_0-auc:0.946639	validation_1-auc:0.939395	validation_2-auc:0.940408
[207]	validation_0-auc:0.94668	validation_1-auc:0.939429	validation_2-auc:0.940443

[294]	validation_0-auc:0.953387	validation_1-auc:0.94499	validation_2-auc:0.946037
[295]	validation_0-auc:0.95342	validation_1-auc:0.945012	validation_2-auc:0.946051
[296]	validation_0-auc:0.953454	validation_1-auc:0.945031	validation_2-auc:0.946067
[297]	validation_0-auc:0.953516	validation_1-auc:0.945077	validation_2-auc:0.946115
[298]	validation_0-auc:0.953531	validation_1-auc:0.945087	validation_2-auc:0.946121
[299]	validation_0-auc:0.95356	validation_1-auc:0.945106	validation_2-auc:0.946142
[300]	validation_0-auc:0.95368	validation_1-auc:0.945197	validation_2-auc:0.946228
[301]	validation_0-auc:0.953701	validation_1-auc:0.945218	validation_2-auc:0.946245
[302]	validation_0-auc:0.953715	validation_1-auc:0.945228	validation_2-auc:0.946252
[303]	validation_0-auc:0.953735	validation_1-auc:0.945226	validation_2-auc:0.946251
[304]	validation_0-auc:0.95377	validation_1-auc:0.945239	validation_2-auc:0.946261
[305]	validation_0-auc:0.95389	validation_1-auc:0.945348	validation_2-auc:0.94636

[392]	validation_0-auc:0.957832	validation_1-auc:0.948326	validation_2-auc:0.949323
[393]	validation_0-auc:0.957848	validation_1-auc:0.94833	validation_2-auc:0.949326
[394]	validation_0-auc:0.957852	validation_1-auc:0.948334	validation_2-auc:0.949327
[395]	validation_0-auc:0.95799	validation_1-auc:0.948464	validation_2-auc:0.949456
[396]	validation_0-auc:0.958003	validation_1-auc:0.948466	validation_2-auc:0.949459
[397]	validation_0-auc:0.958045	validation_1-auc:0.948493	validation_2-auc:0.94949
[398]	validation_0-auc:0.958126	validation_1-auc:0.948559	validation_2-auc:0.949557
[399]	validation_0-auc:0.958218	validation_1-auc:0.948641	validation_2-auc:0.949631
[400]	validation_0-auc:0.958273	validation_1-auc:0.948678	validation_2-auc:0.949664
[401]	validation_0-auc:0.958297	validation_1-auc:0.948685	validation_2-auc:0.949664
[402]	validation_0-auc:0.958418	validation_1-auc:0.948784	validation_2-auc:0.949769
[403]	validation_0-auc:0.958442	validation_1-auc:0.948798	validation_2-auc:0.94

...

[588]	validation_0-auc:0.964859	validation_1-auc:0.953107	validation_2-auc:0.953996
[589]	validation_0-auc:0.964875	validation_1-auc:0.953116	validation_2-auc:0.954004
[590]	validation_0-auc:0.964883	validation_1-auc:0.953115	validation_2-auc:0.954005
[591]	validation_0-auc:0.964893	validation_1-auc:0.953113	validation_2-auc:0.954004
[592]	validation_0-auc:0.964908	validation_1-auc:0.953116	validation_2-auc:0.954006
[593]	validation_0-auc:0.96493	validation_1-auc:0.953115	validation_2-auc:0.954004
[594]	validation_0-auc:0.964939	validation_1-auc:0.953114	validation_2-auc:0.954004
[595]	validation_0-auc:0.965047	validation_1-auc:0.953221	validation_2-auc:0.954102
[596]	validation_0-auc:0.965095	validation_1-auc:0.953254	validation_2-auc:0.954136
[597]	validation_0-auc:0.965151	validation_1-auc:0.9533	validation_2-auc:0.954181
[598]	validation_0-auc:0.965174	validation_1-auc:0.953306	validation_2-auc:0.954191
[599]	validation_0-auc:0.965196	validation_1-auc:0.95332	validation_2-auc:0.954

[686]	validation_0-auc:0.967642	validation_1-auc:0.95473	validation_2-auc:0.955594
[687]	validation_0-auc:0.967672	validation_1-auc:0.954746	validation_2-auc:0.955603
[688]	validation_0-auc:0.967676	validation_1-auc:0.954747	validation_2-auc:0.955603
[689]	validation_0-auc:0.967683	validation_1-auc:0.954747	validation_2-auc:0.955606
[690]	validation_0-auc:0.96769	validation_1-auc:0.95475	validation_2-auc:0.955605
[691]	validation_0-auc:0.967709	validation_1-auc:0.954748	validation_2-auc:0.955612
[692]	validation_0-auc:0.967765	validation_1-auc:0.954796	validation_2-auc:0.955657
[693]	validation_0-auc:0.967782	validation_1-auc:0.954794	validation_2-auc:0.955655
[694]	validation_0-auc:0.967796	validation_1-auc:0.954797	validation_2-auc:0.955658
[695]	validation_0-auc:0.967857	validation_1-auc:0.954846	validation_2-auc:0.9557
[696]	validation_0-auc:0.967881	validation_1-auc:0.954866	validation_2-auc:0.955716
[697]	validation_0-auc:0.967929	validation_1-auc:0.954901	validation_2-auc:0.9557

[784]	validation_0-auc:0.970067	validation_1-auc:0.956217	validation_2-auc:0.957069
[785]	validation_0-auc:0.970075	validation_1-auc:0.956217	validation_2-auc:0.957072
[786]	validation_0-auc:0.970084	validation_1-auc:0.956212	validation_2-auc:0.957069
[787]	validation_0-auc:0.970092	validation_1-auc:0.956211	validation_2-auc:0.95707
[788]	validation_0-auc:0.970143	validation_1-auc:0.956254	validation_2-auc:0.957115
[789]	validation_0-auc:0.970197	validation_1-auc:0.956302	validation_2-auc:0.957162
[790]	validation_0-auc:0.970206	validation_1-auc:0.956307	validation_2-auc:0.957162
[791]	validation_0-auc:0.970218	validation_1-auc:0.956307	validation_2-auc:0.957169
[792]	validation_0-auc:0.97026	validation_1-auc:0.956342	validation_2-auc:0.957196
[793]	validation_0-auc:0.970267	validation_1-auc:0.956341	validation_2-auc:0.957193
[794]	validation_0-auc:0.970285	validation_1-auc:0.956351	validation_2-auc:0.9572
[795]	validation_0-auc:0.970297	validation_1-auc:0.956353	validation_2-auc:0.957

[882]	validation_0-auc:0.972026	validation_1-auc:0.957205	validation_2-auc:0.958037
[883]	validation_0-auc:0.972035	validation_1-auc:0.957211	validation_2-auc:0.95804
[884]	validation_0-auc:0.972048	validation_1-auc:0.957211	validation_2-auc:0.958037
[885]	validation_0-auc:0.972073	validation_1-auc:0.957216	validation_2-auc:0.958043
[886]	validation_0-auc:0.972091	validation_1-auc:0.957217	validation_2-auc:0.958047
[887]	validation_0-auc:0.972113	validation_1-auc:0.957214	validation_2-auc:0.958041
[888]	validation_0-auc:0.972119	validation_1-auc:0.957212	validation_2-auc:0.958041
[889]	validation_0-auc:0.972136	validation_1-auc:0.957213	validation_2-auc:0.958041
[890]	validation_0-auc:0.972144	validation_1-auc:0.957214	validation_2-auc:0.958042
[891]	validation_0-auc:0.972156	validation_1-auc:0.957217	validation_2-auc:0.958043
[892]	validation_0-auc:0.972164	validation_1-auc:0.957215	validation_2-auc:0.958042
[893]	validation_0-auc:0.972174	validation_1-auc:0.957217	validation_2-auc:0.

69

### XGB approx

In [27]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'approx'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.778673	validation_1-auc:0.778255	validation_2-auc:0.776844
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.806723	validation_1-auc:0.805743	validation_2-auc:0.804216
[2]	validation_0-auc:0.820539	validation_1-auc:0.819219	validation_2-auc:0.818126
[3]	validation_0-auc:0.832858	validation_1-auc:0.83127	validation_2-auc:0.830623
[4]	validation_0-auc:0.838434	validation_1-auc:0.836744	validation_2-auc:0.836065
[5]	validation_0-auc:0.841453	validation_1-auc:0.839702	validation_2-auc:0.838978
[6]	validation_0-auc:0.842951	validation_1-auc:0.841313	validation_2-auc:0.840354
[7]	validation_0-auc:0.848123	validation_1-auc:0.84633	validation_2-auc:0.845386
[8]	validation_0-auc:0.852353	validation_1-auc:0.850536	validation_2-auc:0.849474
[9]	validation_0-auc:0.855367	validation_1-auc:0.853504	validation_2-auc:0.852458
[10]	validation_0-auc:0.8

[98]	validation_0-auc:0.931875	validation_1-auc:0.92752	validation_2-auc:0.927906
[99]	validation_0-auc:0.932016	validation_1-auc:0.927636	validation_2-auc:0.928034
[100]	validation_0-auc:0.932296	validation_1-auc:0.927907	validation_2-auc:0.928303
[101]	validation_0-auc:0.932578	validation_1-auc:0.928184	validation_2-auc:0.928577
[102]	validation_0-auc:0.932672	validation_1-auc:0.928255	validation_2-auc:0.92865
[103]	validation_0-auc:0.933094	validation_1-auc:0.928685	validation_2-auc:0.929067
[104]	validation_0-auc:0.933404	validation_1-auc:0.928982	validation_2-auc:0.929346
[105]	validation_0-auc:0.933832	validation_1-auc:0.929407	validation_2-auc:0.929753
[106]	validation_0-auc:0.934223	validation_1-auc:0.929764	validation_2-auc:0.930115
[107]	validation_0-auc:0.934347	validation_1-auc:0.92986	validation_2-auc:0.930224
[108]	validation_0-auc:0.934566	validation_1-auc:0.930043	validation_2-auc:0.930423
[109]	validation_0-auc:0.934745	validation_1-auc:0.930197	validation_2-auc:0.9305

[196]	validation_0-auc:0.950349	validation_1-auc:0.944297	validation_2-auc:0.944918
[197]	validation_0-auc:0.950408	validation_1-auc:0.944327	validation_2-auc:0.944945
[198]	validation_0-auc:0.950479	validation_1-auc:0.944384	validation_2-auc:0.945002
[199]	validation_0-auc:0.950577	validation_1-auc:0.944463	validation_2-auc:0.945075
[200]	validation_0-auc:0.950798	validation_1-auc:0.944673	validation_2-auc:0.945277
[201]	validation_0-auc:0.950897	validation_1-auc:0.944757	validation_2-auc:0.945361
[202]	validation_0-auc:0.950952	validation_1-auc:0.944795	validation_2-auc:0.94539
[203]	validation_0-auc:0.951258	validation_1-auc:0.945101	validation_2-auc:0.945687
[204]	validation_0-auc:0.951323	validation_1-auc:0.945148	validation_2-auc:0.94575
[205]	validation_0-auc:0.951402	validation_1-auc:0.945199	validation_2-auc:0.945799
[206]	validation_0-auc:0.95155	validation_1-auc:0.945338	validation_2-auc:0.945918
[207]	validation_0-auc:0.951684	validation_1-auc:0.945464	validation_2-auc:0.94

[294]	validation_0-auc:0.958905	validation_1-auc:0.951204	validation_2-auc:0.951814
[295]	validation_0-auc:0.958958	validation_1-auc:0.951245	validation_2-auc:0.951864
[296]	validation_0-auc:0.959048	validation_1-auc:0.951316	validation_2-auc:0.951944
[297]	validation_0-auc:0.959173	validation_1-auc:0.951425	validation_2-auc:0.95206
[298]	validation_0-auc:0.959222	validation_1-auc:0.951453	validation_2-auc:0.952092
[299]	validation_0-auc:0.959275	validation_1-auc:0.951493	validation_2-auc:0.952135
[300]	validation_0-auc:0.959334	validation_1-auc:0.951538	validation_2-auc:0.952184
[301]	validation_0-auc:0.959422	validation_1-auc:0.951595	validation_2-auc:0.952243
[302]	validation_0-auc:0.959492	validation_1-auc:0.951652	validation_2-auc:0.952289
[303]	validation_0-auc:0.959538	validation_1-auc:0.951677	validation_2-auc:0.952315
[304]	validation_0-auc:0.959628	validation_1-auc:0.951738	validation_2-auc:0.952392
[305]	validation_0-auc:0.95966	validation_1-auc:0.951753	validation_2-auc:0.9

[392]	validation_0-auc:0.964284	validation_1-auc:0.954897	validation_2-auc:0.955665
[393]	validation_0-auc:0.964312	validation_1-auc:0.9549	validation_2-auc:0.955676
[394]	validation_0-auc:0.964384	validation_1-auc:0.954955	validation_2-auc:0.955732
[395]	validation_0-auc:0.964441	validation_1-auc:0.954991	validation_2-auc:0.955783
[396]	validation_0-auc:0.964486	validation_1-auc:0.955025	validation_2-auc:0.955812
[397]	validation_0-auc:0.964522	validation_1-auc:0.955044	validation_2-auc:0.955829
[398]	validation_0-auc:0.964546	validation_1-auc:0.955044	validation_2-auc:0.95583
[399]	validation_0-auc:0.964596	validation_1-auc:0.955074	validation_2-auc:0.955871
[400]	validation_0-auc:0.964633	validation_1-auc:0.955094	validation_2-auc:0.955891
[401]	validation_0-auc:0.964694	validation_1-auc:0.955151	validation_2-auc:0.955937
[402]	validation_0-auc:0.964724	validation_1-auc:0.95516	validation_2-auc:0.955949
[403]	validation_0-auc:0.964754	validation_1-auc:0.95518	validation_2-auc:0.9559

...

[686]	validation_0-auc:0.974065	validation_1-auc:0.96028	validation_2-auc:0.960931
[687]	validation_0-auc:0.974087	validation_1-auc:0.960282	validation_2-auc:0.960931
[688]	validation_0-auc:0.974112	validation_1-auc:0.960284	validation_2-auc:0.960933
[689]	validation_0-auc:0.974133	validation_1-auc:0.960285	validation_2-auc:0.960936
[690]	validation_0-auc:0.974169	validation_1-auc:0.960311	validation_2-auc:0.960963
[691]	validation_0-auc:0.974186	validation_1-auc:0.960313	validation_2-auc:0.96096
[692]	validation_0-auc:0.974213	validation_1-auc:0.960331	validation_2-auc:0.960971
[693]	validation_0-auc:0.974236	validation_1-auc:0.960353	validation_2-auc:0.960991
[694]	validation_0-auc:0.974253	validation_1-auc:0.960357	validation_2-auc:0.960999
[695]	validation_0-auc:0.974278	validation_1-auc:0.960367	validation_2-auc:0.96101
[696]	validation_0-auc:0.974304	validation_1-auc:0.960372	validation_2-auc:0.961009
[697]	validation_0-auc:0.974321	validation_1-auc:0.96038	validation_2-auc:0.961

[784]	validation_0-auc:0.976401	validation_1-auc:0.961239	validation_2-auc:0.961845
[785]	validation_0-auc:0.976422	validation_1-auc:0.961254	validation_2-auc:0.961858
[786]	validation_0-auc:0.976441	validation_1-auc:0.961265	validation_2-auc:0.961864
[787]	validation_0-auc:0.976463	validation_1-auc:0.961272	validation_2-auc:0.961869
[788]	validation_0-auc:0.976488	validation_1-auc:0.961279	validation_2-auc:0.961879
[789]	validation_0-auc:0.976515	validation_1-auc:0.961295	validation_2-auc:0.961886
[790]	validation_0-auc:0.976535	validation_1-auc:0.961311	validation_2-auc:0.961892
[791]	validation_0-auc:0.976554	validation_1-auc:0.961307	validation_2-auc:0.961895
[792]	validation_0-auc:0.976588	validation_1-auc:0.961328	validation_2-auc:0.961919
[793]	validation_0-auc:0.976605	validation_1-auc:0.961335	validation_2-auc:0.961923
[794]	validation_0-auc:0.976628	validation_1-auc:0.961348	validation_2-auc:0.961932
[795]	validation_0-auc:0.976654	validation_1-auc:0.961365	validation_2-auc:0

[882]	validation_0-auc:0.97852	validation_1-auc:0.962061	validation_2-auc:0.962642
[883]	validation_0-auc:0.978538	validation_1-auc:0.962064	validation_2-auc:0.962644
[884]	validation_0-auc:0.97856	validation_1-auc:0.962075	validation_2-auc:0.962655
[885]	validation_0-auc:0.978577	validation_1-auc:0.962081	validation_2-auc:0.962662
[886]	validation_0-auc:0.978589	validation_1-auc:0.962082	validation_2-auc:0.962665
[887]	validation_0-auc:0.978608	validation_1-auc:0.962081	validation_2-auc:0.962661
[888]	validation_0-auc:0.978627	validation_1-auc:0.962078	validation_2-auc:0.962662
[889]	validation_0-auc:0.978638	validation_1-auc:0.962073	validation_2-auc:0.962659
[890]	validation_0-auc:0.978654	validation_1-auc:0.962072	validation_2-auc:0.962655
[891]	validation_0-auc:0.978677	validation_1-auc:0.962081	validation_2-auc:0.962662
[892]	validation_0-auc:0.978698	validation_1-auc:0.962097	validation_2-auc:0.962676
[893]	validation_0-auc:0.978712	validation_1-auc:0.962096	validation_2-auc:0.9

[980]	validation_0-auc:0.980273	validation_1-auc:0.962596	validation_2-auc:0.963156
[981]	validation_0-auc:0.980288	validation_1-auc:0.962601	validation_2-auc:0.963155
[982]	validation_0-auc:0.980303	validation_1-auc:0.962604	validation_2-auc:0.963159
[983]	validation_0-auc:0.980322	validation_1-auc:0.962608	validation_2-auc:0.963162
[984]	validation_0-auc:0.980341	validation_1-auc:0.962616	validation_2-auc:0.963172
[985]	validation_0-auc:0.980355	validation_1-auc:0.962612	validation_2-auc:0.963167
[986]	validation_0-auc:0.980373	validation_1-auc:0.962615	validation_2-auc:0.963173
[987]	validation_0-auc:0.980392	validation_1-auc:0.962624	validation_2-auc:0.96318
[988]	validation_0-auc:0.980408	validation_1-auc:0.962626	validation_2-auc:0.963181
[989]	validation_0-auc:0.980423	validation_1-auc:0.962627	validation_2-auc:0.963181
[990]	validation_0-auc:0.980441	validation_1-auc:0.962631	validation_2-auc:0.963181
[991]	validation_0-auc:0.980459	validation_1-auc:0.962628	validation_2-auc:0.

69

### XGB hist

In [28]:
%%time
params = {
    'learning_rate': .1,
    'max_depth': 7,
    'min_child_weight': 3,
    'subsample': .8,
    'colsample_bytree': .8,
    'reg_lambda': .05,
    'reg_alpha': .025,
    'gamma': .1,
    'objective': 'binary:logistic',
    'n_estimators': 1000,
    'n_jobs': -1,
    'random_state': 0,
    'tree_method': 'hist'
}

xgb = XGBClassifier(**params)

xgb.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test), (X_valid, y_valid)],
        eval_metric='auc',
        verbose=True,
        early_stopping_rounds=10)

del xgb
gc.collect()

[0]	validation_0-auc:0.782079	validation_1-auc:0.779527	validation_2-auc:0.778348
Multiple eval metrics have been passed: 'validation_2-auc' will be used for early stopping.

Will train until validation_2-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.815314	validation_1-auc:0.812274	validation_2-auc:0.811527
[2]	validation_0-auc:0.828564	validation_1-auc:0.825875	validation_2-auc:0.824549
[3]	validation_0-auc:0.832599	validation_1-auc:0.829789	validation_2-auc:0.828945
[4]	validation_0-auc:0.8389	validation_1-auc:0.836098	validation_2-auc:0.83518
[5]	validation_0-auc:0.839674	validation_1-auc:0.836676	validation_2-auc:0.835989
[6]	validation_0-auc:0.840714	validation_1-auc:0.837613	validation_2-auc:0.837034
[7]	validation_0-auc:0.851025	validation_1-auc:0.847892	validation_2-auc:0.847244
[8]	validation_0-auc:0.85549	validation_1-auc:0.852486	validation_2-auc:0.851752
[9]	validation_0-auc:0.859964	validation_1-auc:0.85721	validation_2-auc:0.856313
[10]	validation_0-auc:0.8661

[98]	validation_0-auc:0.9274	validation_1-auc:0.922405	validation_2-auc:0.92305
[99]	validation_0-auc:0.92754	validation_1-auc:0.922535	validation_2-auc:0.923185
[100]	validation_0-auc:0.928131	validation_1-auc:0.923112	validation_2-auc:0.923789
[101]	validation_0-auc:0.928194	validation_1-auc:0.923149	validation_2-auc:0.923826
[102]	validation_0-auc:0.928353	validation_1-auc:0.923294	validation_2-auc:0.923966
[103]	validation_0-auc:0.92908	validation_1-auc:0.923987	validation_2-auc:0.924672
[104]	validation_0-auc:0.929566	validation_1-auc:0.924458	validation_2-auc:0.925119
[105]	validation_0-auc:0.929639	validation_1-auc:0.924511	validation_2-auc:0.925181
[106]	validation_0-auc:0.929837	validation_1-auc:0.924682	validation_2-auc:0.925356
[107]	validation_0-auc:0.929997	validation_1-auc:0.924841	validation_2-auc:0.925512
[108]	validation_0-auc:0.930435	validation_1-auc:0.92528	validation_2-auc:0.925964
[109]	validation_0-auc:0.930577	validation_1-auc:0.925412	validation_2-auc:0.926082


[197]	validation_0-auc:0.947264	validation_1-auc:0.940677	validation_2-auc:0.941523
[198]	validation_0-auc:0.94729	validation_1-auc:0.940697	validation_2-auc:0.941544
[199]	validation_0-auc:0.947366	validation_1-auc:0.94075	validation_2-auc:0.941595
[200]	validation_0-auc:0.947406	validation_1-auc:0.940782	validation_2-auc:0.94163
[201]	validation_0-auc:0.947471	validation_1-auc:0.940825	validation_2-auc:0.941674
[202]	validation_0-auc:0.947539	validation_1-auc:0.940871	validation_2-auc:0.94173
[203]	validation_0-auc:0.947622	validation_1-auc:0.940931	validation_2-auc:0.941795
[204]	validation_0-auc:0.947784	validation_1-auc:0.941074	validation_2-auc:0.941934
[205]	validation_0-auc:0.947861	validation_1-auc:0.941135	validation_2-auc:0.942017
[206]	validation_0-auc:0.948096	validation_1-auc:0.941344	validation_2-auc:0.942221
[207]	validation_0-auc:0.948249	validation_1-auc:0.941495	validation_2-auc:0.942365
[208]	validation_0-auc:0.948304	validation_1-auc:0.94153	validation_2-auc:0.9424

[296]	validation_0-auc:0.956525	validation_1-auc:0.948521	validation_2-auc:0.949363
[297]	validation_0-auc:0.956594	validation_1-auc:0.948581	validation_2-auc:0.949422
[298]	validation_0-auc:0.956714	validation_1-auc:0.948699	validation_2-auc:0.94952
[299]	validation_0-auc:0.956768	validation_1-auc:0.948739	validation_2-auc:0.949567
[300]	validation_0-auc:0.956835	validation_1-auc:0.948789	validation_2-auc:0.949623
[301]	validation_0-auc:0.956864	validation_1-auc:0.948799	validation_2-auc:0.949641
[302]	validation_0-auc:0.957012	validation_1-auc:0.948943	validation_2-auc:0.949754
[303]	validation_0-auc:0.957044	validation_1-auc:0.948948	validation_2-auc:0.949747
[304]	validation_0-auc:0.957078	validation_1-auc:0.948951	validation_2-auc:0.949751
[305]	validation_0-auc:0.957149	validation_1-auc:0.949001	validation_2-auc:0.949797
[306]	validation_0-auc:0.957182	validation_1-auc:0.949018	validation_2-auc:0.94981
[307]	validation_0-auc:0.957231	validation_1-auc:0.949056	validation_2-auc:0.9

[394]	validation_0-auc:0.961983	validation_1-auc:0.952492	validation_2-auc:0.953285
[395]	validation_0-auc:0.962009	validation_1-auc:0.952506	validation_2-auc:0.953292
[396]	validation_0-auc:0.962034	validation_1-auc:0.952516	validation_2-auc:0.953301
[397]	validation_0-auc:0.962065	validation_1-auc:0.952536	validation_2-auc:0.953313
[398]	validation_0-auc:0.962084	validation_1-auc:0.952538	validation_2-auc:0.953313
[399]	validation_0-auc:0.962111	validation_1-auc:0.952544	validation_2-auc:0.953322
[400]	validation_0-auc:0.962141	validation_1-auc:0.952568	validation_2-auc:0.953353
[401]	validation_0-auc:0.962179	validation_1-auc:0.952581	validation_2-auc:0.953359
[402]	validation_0-auc:0.962211	validation_1-auc:0.952601	validation_2-auc:0.953372
[403]	validation_0-auc:0.962284	validation_1-auc:0.952657	validation_2-auc:0.953427
[404]	validation_0-auc:0.962325	validation_1-auc:0.952684	validation_2-auc:0.953445
[405]	validation_0-auc:0.962389	validation_1-auc:0.952745	validation_2-auc:0

...

[689]	validation_0-auc:0.972429	validation_1-auc:0.958795	validation_2-auc:0.959482
[690]	validation_0-auc:0.972453	validation_1-auc:0.958804	validation_2-auc:0.959484
[691]	validation_0-auc:0.972483	validation_1-auc:0.958818	validation_2-auc:0.959499
[692]	validation_0-auc:0.972499	validation_1-auc:0.958829	validation_2-auc:0.959507
[693]	validation_0-auc:0.972517	validation_1-auc:0.958829	validation_2-auc:0.959508
[694]	validation_0-auc:0.972553	validation_1-auc:0.958863	validation_2-auc:0.959548
[695]	validation_0-auc:0.972586	validation_1-auc:0.95888	validation_2-auc:0.959575
[696]	validation_0-auc:0.972602	validation_1-auc:0.958884	validation_2-auc:0.95958
[697]	validation_0-auc:0.972625	validation_1-auc:0.958881	validation_2-auc:0.959585
[698]	validation_0-auc:0.972645	validation_1-auc:0.958893	validation_2-auc:0.959592
[699]	validation_0-auc:0.972664	validation_1-auc:0.958898	validation_2-auc:0.959596
[700]	validation_0-auc:0.972683	validation_1-auc:0.958902	validation_2-auc:0.9

[787]	validation_0-auc:0.974956	validation_1-auc:0.959921	validation_2-auc:0.960633
[788]	validation_0-auc:0.974984	validation_1-auc:0.959936	validation_2-auc:0.960655
[789]	validation_0-auc:0.975006	validation_1-auc:0.95994	validation_2-auc:0.960661
[790]	validation_0-auc:0.975029	validation_1-auc:0.959949	validation_2-auc:0.96067
[791]	validation_0-auc:0.975046	validation_1-auc:0.959953	validation_2-auc:0.960675
[792]	validation_0-auc:0.97507	validation_1-auc:0.95995	validation_2-auc:0.960679
[793]	validation_0-auc:0.975092	validation_1-auc:0.959952	validation_2-auc:0.960688
[794]	validation_0-auc:0.975118	validation_1-auc:0.959976	validation_2-auc:0.960709
[795]	validation_0-auc:0.975141	validation_1-auc:0.959987	validation_2-auc:0.960727
[796]	validation_0-auc:0.975157	validation_1-auc:0.959992	validation_2-auc:0.960731
[797]	validation_0-auc:0.975181	validation_1-auc:0.960004	validation_2-auc:0.96074
[798]	validation_0-auc:0.975204	validation_1-auc:0.960005	validation_2-auc:0.9607

[885]	validation_0-auc:0.976972	validation_1-auc:0.960676	validation_2-auc:0.96142
[886]	validation_0-auc:0.976991	validation_1-auc:0.960688	validation_2-auc:0.961434
[887]	validation_0-auc:0.977029	validation_1-auc:0.960718	validation_2-auc:0.961459
[888]	validation_0-auc:0.977046	validation_1-auc:0.96072	validation_2-auc:0.961461
[889]	validation_0-auc:0.977064	validation_1-auc:0.96073	validation_2-auc:0.961467
[890]	validation_0-auc:0.977094	validation_1-auc:0.960744	validation_2-auc:0.961483
[891]	validation_0-auc:0.97712	validation_1-auc:0.960747	validation_2-auc:0.961489
[892]	validation_0-auc:0.977139	validation_1-auc:0.96076	validation_2-auc:0.961503
[893]	validation_0-auc:0.977157	validation_1-auc:0.960764	validation_2-auc:0.961505
[894]	validation_0-auc:0.977173	validation_1-auc:0.96076	validation_2-auc:0.961506
[895]	validation_0-auc:0.977187	validation_1-auc:0.960765	validation_2-auc:0.96151
[896]	validation_0-auc:0.977203	validation_1-auc:0.960764	validation_2-auc:0.961514

[983]	validation_0-auc:0.978788	validation_1-auc:0.961268	validation_2-auc:0.962012
[984]	validation_0-auc:0.978799	validation_1-auc:0.961271	validation_2-auc:0.962013
[985]	validation_0-auc:0.978815	validation_1-auc:0.961272	validation_2-auc:0.962018
[986]	validation_0-auc:0.978832	validation_1-auc:0.961281	validation_2-auc:0.962029
[987]	validation_0-auc:0.978842	validation_1-auc:0.961283	validation_2-auc:0.96203
[988]	validation_0-auc:0.97886	validation_1-auc:0.961292	validation_2-auc:0.962036
[989]	validation_0-auc:0.978877	validation_1-auc:0.961295	validation_2-auc:0.96204
[990]	validation_0-auc:0.978889	validation_1-auc:0.9613	validation_2-auc:0.962038
[991]	validation_0-auc:0.978902	validation_1-auc:0.961297	validation_2-auc:0.962041
[992]	validation_0-auc:0.978918	validation_1-auc:0.961306	validation_2-auc:0.962045
[993]	validation_0-auc:0.97894	validation_1-auc:0.961314	validation_2-auc:0.962048
[994]	validation_0-auc:0.978954	validation_1-auc:0.961315	validation_2-auc:0.96204

69

In [29]:
del X_train, y_train, X_valid, y_valid, X_test, y_test
gc.collect()

0